In [ ]:
import os
import time
import numpy as np
from glob import glob
import json
import random
import cv2
import gc
from time import time
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'
#os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import tensorflow as tf
policy = tf.keras.mixed_precision.Policy("mixed_float16")
tf.keras.mixed_precision.set_global_policy(policy)
strategy = tf.distribute.MultiWorkerMirroredStrategy()
from tensorflow import keras
from tensorflow.keras import layers
from keras_cv_attention_models import efficientnet

In [ ]:
image_frames = 60
frame_draw_size = 384
input_shape = (image_frames, frame_draw_size, frame_draw_size, 3)

In [ ]:

with strategy.scope():
    models = glob('models/*.h5')
    if len(models) > 0:
        models.sort(key=os.path.getmtime)
        model_name = models[-1]
        print('loading model: ' + model_name)
        model = keras.models.load_model(model_name)
        model.summary()
    else:
        inputs = keras.Input(shape=(image_frames, 12, 12, 1280))
        boneless_inputs = keras.Input(shape=(12, 12, 1280))
        y = layers.Flatten()(boneless_inputs)
        y = layers.Dense(64, activation="relu")(y)
        y = layers.Dropout(0.001)(y)
        x = layers.TimeDistributed(keras.Model(boneless_inputs, y))(inputs)
        x = layers.Dropout(0.001)(x)
        x = layers.Bidirectional(layers.LSTM(128, return_sequences=True))(x)
        x = layers.Dropout(0.001)(x)
        x = layers.Bidirectional(layers.LSTM(128, return_sequences=False))(x)
        x = layers.Dropout(0.001)(x)
        x = layers.Dense(image_frames * 6 * 2, activation="relu")(x)
        x = layers.Dropout(0.001)(x)
        # x = layers.Dense(128, activation="relu")(x)
        # x = layers.Dropout(0.001)(x)
        # x = layers.Dense(64, activation="relu")(x)
        # x = layers.Dropout(0.1)(x)
        # x = layers.Dense(64, activation="relu")(x)
        # x = layers.Dropout(0.1)(x)
        # x = layers.Dense(64, activation="relu")(x)
        # x = layers.Dropout(0.1)(x)
        outputs = layers.Dense(image_frames * 6, activation="relu")(x)
        model = keras.Model(inputs, outputs)
        print(model.summary())
        model.compile(
            optimizer=keras.optimizers.Adam(1e-3),
            loss="mean_squared_error",
            metrics=["mean_squared_error", "mean_absolute_error"]
        )
        model.save('models/base_' + str(image_frames) + "_" + str(frame_draw_size) + '_12_12_1280_short_thicc.h5')



In [ ]:
import numpy as np
import os
import random
from glob import glob

data_sequence_folder = './dataset'
data_sequence_files = os.listdir(data_sequence_folder)
# filter out non .npy files
data_sequence_files = [f for f in data_sequence_files if f.endswith('.npy')]
# split all the filenames by _frame_ and keep the first part
# then remove the duplicates
data_sequence_names = list(set([f.split('_frame_')[0] for f in data_sequence_files]))
print(data_sequence_names)

data_sequences = []
for data_sequence_name in data_sequence_names:
    #print(data_sequence_name)
    print(data_sequence_name)
    sequence_data_sequence_files = [f for f in data_sequence_files if data_sequence_name in f]
    print(len(sequence_data_sequence_files))
    # sort the files by the frame number which is the last part of the filename
    sequence_data_sequence_files.sort(key=lambda x: int(x.split('_frame_')[1].split('.')[0]))
    # find sequencial frames
    data_sequence = []
    for i in range(len(sequence_data_sequence_files) - 1):
        if int(sequence_data_sequence_files[i].split('_frame_')[1].split('.')[0]) + 1 == int(sequence_data_sequence_files[i+1].split('_frame_')[1].split('.')[0]):
            data_sequence.append(sequence_data_sequence_files[i])
        else:
            data_sequence.append(sequence_data_sequence_files[i])
            #print(data_sequence)
            data_sequences.append(data_sequence)
            data_sequence = []
    data_sequence.append(sequence_data_sequence_files[-1])
    data_sequences.append(data_sequence)
print(len(data_sequences))

# drop sequences that are too short
data_sequences = [s for s in data_sequences if len(s) > 60]

print("Loaded {} sequences".format(len(data_sequences)))

def get_sequences(data_sequences_, results):
    random_data_sequences_ = []
    random_label_sequences_ = []
    for i in range(results):
        random_data_sequence = random.choice(data_sequences)
        random_start = random.randint(0, len(random_data_sequence) - 60)
        random_sequence = random_data_sequence[random_start:random_start+60]
        random_sequence_data = []
        random_sequence_label = []
        for j in random_sequence:
            random_sequence_data.append(np.load(os.path.join(data_sequence_folder, j)))
            label_file = j.replace('.npy', '.txt')
            with open(os.path.join(data_sequence_folder, label_file)) as f:
                sequence_label = f.read().strip()
            sequence_label = [float (i) for i in sequence_label.split('\n')]
            random_sequence_label.extend(sequence_label)
        random_sequence_label = np.array(random_sequence_label)
        random_sequence_data = np.array(random_sequence_data)
        random_data_sequences_.append(random_sequence_data)
        random_label_sequences_.append(random_sequence_label)
    random_data_sequences_ = np.array(random_data_sequences_)
    random_label_sequences_ = np.array(random_label_sequences_)

    return [random_data_sequences_, random_label_sequences_]

random_data_sequences = get_sequences(data_sequences, 128)
for s in random_data_sequences:
    print(s.shape)


In [ ]:
epoch = 0
lr = 1e-3


In [ ]:

### TRAINING LOOP RUNS FOREVER
with strategy.scope():
    while True:
        try:
            random_data_sequences = get_sequences(data_sequences, 256)  # THIS USES ALOT OF MEMORY
            # set learning rate
            model.optimizer.learning_rate = lr
            print("Learning rate: ", lr)
            lr = lr * 0.9

            tensorboard_callback = keras.callbacks.TensorBoard(log_dir="logs/{}".format(time()))
            
            callbacks = [tensorboard_callback]
            gc.collect()
            epochs = epoch + 100
            model.fit(random_data_sequences[0], random_data_sequences[1], batch_size=4, epochs=epochs, callbacks = callbacks, initial_epoch=epoch, verbose=1)
            epoch = epoch + 100
        except Exception as e:
            print(e)
            break

In [ ]:
model.save('models/trained_from_features_' + str(image_frames) + "_" + str(frame_draw_size) + '_12_12_1280_shortthicc_' + str(epoch) + '.h5')
model.save('models/trained_from_features_' + str(image_frames) + "_" + str(frame_draw_size) + '_12_12_1280_shortthicc_' + str(epoch) + '.keras')
model.save('models/trained_from_features_' + str(image_frames) + "_" + str(frame_draw_size) + '_12_12_1280_shortthicc_' + str(epoch) + '.tf')